<a href="https://colab.research.google.com/github/jing-zhou/w2v/blob/master/w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Environment setup and library import**

In [1]:
import random                           # Generates random numbers
import os                               # Create directories, list files
from zipfile import ZipFile
from shutil import copyfile             # Copy files from Source to Destination
from time import time
import cv2                              # To flip images in data augmentation
import matplotlib.pyplot as plt         # To save the images
import matplotlib.image as mpimg

!pip install Cython

**2. Link Goggle Drive with Colab**

Source : https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d

In [2]:
# Check is Link to Drive is OK
google = !if [ -d 'GDrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] == '0' ):
  from google.colab import drive
  drive.mount('/content/GDrive/')
!if [ -d 'GDrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi


Mounted at /content/GDrive/
Connection to Google drive successful


**3. Train W2V with Gensim**

In [3]:
from gensim.models.word2vec import PathLineSentences
import multiprocessing
import gensim.models

class MyCorpus:
  def __init__(self, corpus_path):
    self.cor_path = corpus_path
    self.cor_list = os.listdir(corpus_path)
  """An interator that yields sentences (lists of str)."""

  def __iter__(self):
    for name in self.cor_list:
      # filter hide file
      if name.startswith('.'):
        continue

      print(name)
      for line in PathLineSentences(self.cor_path + '/' + name):
        # print(line)
        # assume there's one document per line, tokens separated by whitespace
        yield line
corpus_dir = "GDrive/MyDrive/corpus/segmented"
sentences = MyCorpus(corpus_dir)
model = gensim.models.Word2Vec(
    sentences=sentences,
    min_count=5,
    vector_size=512,
    compute_loss=True,
    workers=multiprocessing.cpu_count())


seg_bxab_1600420891674
seg_dxaa_1600420892913
seg_cxab_1600420894008
seg_dxaj_1600420894978
seg_fxaa_1600420895595
seg_cxal_1600420896674
seg_exae_1600420897586
seg_bxae_1600420898825
seg_exag_1600420899885
seg_bxac_1600420901018
seg_axal_1600420902316
seg_dxai_1600420903234
seg_exaf_1600420903664
seg_exaj_1600420904714
seg_axaa_1600420905944
seg_axar_1600420907343
seg_axag_1600420908031
seg_fxac_1600420909006
seg_bxad_1600420910165
seg_cxae_1600420911294
seg_cxai_1600420912337
seg_exab_1600420913338
seg_exaa_1600420913771
seg_dxag_1600420914627
seg_bxaa_1600420915406
seg_cxag_1600420916450
seg_cxaa_1600420917469
seg_axac_1600420918291
seg_bxak_1600420919325
seg_bxal_1600420920287
seg_dxah_1600420921374
seg_axai_1600420922073
seg_dxae_1600420923040
seg_exah_1600420923948
seg_exac_1600420925168
seg_fxah_1600420925840
seg_cxaj_1600420926697
seg_bxai_1600420927726
seg_dxad_1600420928760
seg_bxaf_1600420929255
seg_bxaj_1600420930323
seg_cxad_1600420931322
seg_fxab_1600420932242
seg_axaj_16

**4. display the W2V**

In [11]:
wv=model.wv
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/828240 is 於
word #1/828240 is 為
word #2/828240 is 後
word #3/828240 is 人口
word #4/828240 is 中国
word #5/828240 is 個
word #6/828240 is 次
word #7/828240 is 名
word #8/828240 is 新
word #9/828240 is 公司


**5. Store the model file**

In [5]:

models_dir = "GDrive/MyDrive/corpus/models/"
if not os.path.exists(models_dir):
  os.mkdir(models_dir)

model.save(models_dir + 'model_' + str(int(time()*1000)))


**7. Demo**

In [20]:

print(model.wv)
print(model.wv['中華民國'])
print(model.wv['中華民國'])

KeyedVectors<vector_size=512, 828240 keys>
[ 1.24284422e+00 -2.77816862e-01 -1.21950054e+00  1.02248514e+00
 -4.01591867e-01  2.36116037e-01  6.33592546e-01  5.29736936e-01
  8.19234133e-01  7.78762341e-01  1.27334368e+00  3.69630963e-01
 -5.07271647e-01  1.99603677e+00 -9.76088643e-01 -8.96043658e-01
  1.80151212e+00 -1.16355109e+00 -7.53009379e-01  6.38366997e-01
  5.54622412e-01  4.14712459e-01 -3.27947408e-01  1.33960843e+00
 -3.08489148e-02 -3.48002136e-01 -3.85625358e-03 -1.13877678e+00
 -1.30254960e+00  1.37084797e-02 -3.93768102e-01 -1.41639411e+00
  4.39297795e-01 -1.80042922e+00 -8.61915767e-01 -1.00454360e-01
  2.85422027e-01  1.36199427e+00  4.89531308e-01 -1.74117613e+00
 -1.34891272e-01  1.59739244e+00 -2.44140792e+00 -4.04287189e-01
  4.97394472e-01 -3.94607008e-01 -1.51772082e+00 -1.97432005e+00
  4.15668398e-01  4.62596193e-02  8.42147768e-01 -4.71452236e-01
 -1.17067122e+00  1.64466679e+00 -1.12797511e+00  7.23381191e-02
  1.23620260e+00  2.05616578e-02  1.11309326e+0